<a href="https://colab.research.google.com/github/NabilNkhili/Doc_Structur-s/blob/main/DEEP_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets

In [4]:
import datasets
print("datasets installé avec succès !")

datasets installé avec succès !


In [5]:
from datasets import load_dataset
import pandas as pd

# Charger le dataset complet
# load_dataset permet d'accéder directement aux datasets hébergés sur Hugging Face Hub.
dataset = load_dataset("Nicolas-BZRD/English_French_Songs_Lyrics_Translation_Original", split="train").to_pandas()

# Créer un sous-ensemble (par exemple 500 lignes aléatoires)
small_dataset = dataset.sample(n=500, random_state=42)

# Sauvegarder ce sous-dataset pour éviter de le recharger
small_dataset.to_csv("small_lyrics_dataset.csv", index=False)

print(small_dataset.head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

(…)-00000-of-00001-667351d80f76bb91.parquet:   0%|          | 0.00/122M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/99289 [00:00<?, ? examples/s]

                artist_name                      album_name    year  \
75903         Emily Loizeau           L'autre Bout Du Monde  2006.0   
12794      Barbra Streisand  The Essential Barbra Streisand  2002.0   
5104   Bring Me The Horizon                     Sempiternal  2013.0   
60578           GloryHammer   Return to the Kingdom of Fife  2023.0   
75032               The Sun           Blame It On The Youth  2005.0   

                                   title  number  \
75903                    Je Suis Jalouse     8.0   
12794                I've Dreamed Of You    16.0   
5104                       Shadows Moses     6.0   
60578  Vorpal Laserblaster of Pittenweem     8.0   
75032                     Romantic Death     0.0   

                                        original_version  \
75903                                               None   
12794  I've dreamed of you,\nAlways feeling you were ...   
5104   Shadow Moses\nCan you tell from the look in ou...   
60578  Long ago in t

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Exemple de texte tokenisé pour la démonstration
X_train = ["chorus Just let me hear", "All through the day I me mine"]
y_train = ["Laisse moi écouter", "Tout le jour je m’appartiens"]

# Tokenisation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train + y_train)
X_seq = tokenizer.texts_to_sequences(X_train)
y_seq = tokenizer.texts_to_sequences(y_train)

# Padding
X_padded = pad_sequences(X_seq, padding="post")
y_padded = pad_sequences(y_seq, padding="post")

# Définir le modèle
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128))
model.add(GRU(256, return_sequences=True))
model.add(Dense(len(tokenizer.word_index) + 1, activation="softmax"))

# Compilation
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Charger le sous-ensemble sauvegardé
small_dataset = pd.read_csv("small_lyrics_dataset.csv")

# Séparation des colonnes
X_train = small_dataset['original_version'].astype(str).tolist()  # Texte en anglais
y_train = small_dataset['french_version'].astype(str).tolist()    # Texte en français

# Tokenisation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train + y_train)

X_seq = tokenizer.texts_to_sequences(X_train)
y_seq = tokenizer.texts_to_sequences(y_train)

# Padding des séquences
X_padded = pad_sequences(X_seq, padding="post")
y_padded = pad_sequences(y_seq, padding="post")

# Définir le modèle GRU
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X_padded.shape[1]))
model.add(GRU(256, return_sequences=True))
model.add(Dense(len(tokenizer.word_index) + 1, activation="softmax"))

# Compilation du modèle
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Affichage de la structure du modèle
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Définir le modèle GRU ajusté
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=X_padded.shape[1]))
model.add(GRU(256, return_sequences=False))  # Pas de retour de séquences
model.add(Dense(len(tokenizer.word_index) + 1, activation="softmax"))

# Compilation
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraîner le modèle
history = model.fit(
    X_padded,
    y_padded[:, 0],  # Prendre seulement la première colonne cible (par exemple pour sortie unique)
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.0698 - loss: 9.6846 - val_accuracy: 0.0800 - val_loss: 8.3935
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.1226 - loss: 6.5539 - val_accuracy: 0.0800 - val_loss: 8.0150
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.0944 - loss: 5.1993 - val_accuracy: 0.0800 - val_loss: 8.7452
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.1202 - loss: 4.8224 - val_accuracy: 0.0800 - val_loss: 9.1342
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.1308 - loss: 4.7835 - val_accuracy: 0.0800 - val_loss: 9.2644
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.1181 - loss: 4.7597 - val_accuracy: 0.0800 - val_loss: 9.3698
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.1268 - loss: 4.7514 - val_accuracy: 0.0800 - val_loss: 9.4098
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.1093 - loss: 4.7836 - val_accuracy: 0.0800 - val_loss:

In [12]:
# Vérifier la forme des cibles
print("Forme avant ajustement :", y_padded.shape)

# Enlever la dimension ajoutée précédemment si elle existe
y_padded = np.squeeze(y_padded)

print("Forme après ajustement :", y_padded.shape)


Forme avant ajustement : (500, 875, 1)
Forme après ajustement : (500, 875)


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Uniformiser la longueur de padding
max_length = max(X_padded.shape[1], y_padded.shape[1])

# Padding des entrées
X_padded = pad_sequences(X_seq, maxlen=max_length, padding="post")

# Padding des cibles
y_padded = pad_sequences(y_seq, maxlen=max_length, padding="post")

print("Nouvelle forme X_padded :", X_padded.shape)
print("Nouvelle forme y_padded :", y_padded.shape)


Nouvelle forme X_padded : (500, 875)
Nouvelle forme y_padded : (500, 875)


In [15]:
# Définir le modèle
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(GRU(256, return_sequences=True))
model.add(Dense(len(tokenizer.word_index) + 1, activation="softmax"))

# Compilation
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraîner le modèle
history = model.fit(
    X_padded,
    y_padded,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 495s 38s/step - accuracy: 0.5494 - loss: 8.7859 - val_accuracy: 0.6990 - val_loss: 4.5653
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 478s 36s/step - accuracy: 0.7090 - loss: 3.6729 - val_accuracy: 0.6990 - val_loss: 2.7576
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 465s 36s/step - accuracy: 0.7168 - loss: 2.5799 - val_accuracy: 0.6990 - val_loss: 2.6787
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 502s 36s/step - accuracy: 0.7132 - loss: 2.4996 - val_accuracy: 0.6990 - val_loss: 2.6196
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 503s 36s/step - accuracy: 0.7147 - loss: 2.3922 - val_accuracy: 0.6990 - val_loss: 2.5385
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 464s 35s/step - accuracy: 0.7135 - loss: 2.3260 - val_accuracy: 0.6990 - val_loss: 2.4414
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 488s 35s/step - accuracy: 0.7102 - loss: 2.2578 - val_accuracy: 0.6990 - val_loss: 2.3708
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 542s 38s/step - accuracy: 0.7059 - loss: 2.2230 - val_accuracy: 0.

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import datasets
print("datasets installé avec succès !")


datasets installé avec succès !


In [3]:
from datasets import load_dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Charger le dataset complet
dataset = load_dataset("Nicolas-BZRD/English_French_Songs_Lyrics_Translation_Original", split="train").to_pandas()

# Séparation des colonnes
X_train = dataset['original_version'].astype(str).tolist()
y_train = dataset['french_version'].astype(str).tolist()

# Tokenisation
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train + y_train)

X_seq = tokenizer.texts_to_sequences(X_train)
y_seq = tokenizer.texts_to_sequences(y_train)

# Padding
max_length = max(max(len(x) for x in X_seq), max(len(y) for y in y_seq))
X_padded = pad_sequences(X_seq, maxlen=max_length, padding="post")
y_padded = pad_sequences(y_seq, maxlen=max_length, padding="post")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Définir le modèle
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(GRU(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(len(tokenizer.word_index) + 1, activation="softmax"))

# Compilation
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# EarlyStopping pour éviter l'overfitting
early_stop = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
# Installer Hugging Face datasets si nécessaire
!pip install datasets

# Importation des bibliothèques nécessaires
import numpy as np
import pandas as pd
from datasets import load_dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping


In [7]:
# Charger le dataset complet
dataset = load_dataset("Nicolas-BZRD/English_French_Songs_Lyrics_Translation_Original", split="train").to_pandas()

# Afficher un aperçu des données
print("Aperçu des données :")
print(dataset.head())

# Séparer les colonnes d'intérêt
X_train = dataset['original_version'].astype(str).tolist()  # Texte en anglais
y_train = dataset['french_version'].astype(str).tolist()    # Texte en français


Aperçu des données :
       artist_name                                         album_name    year  \
0      The Beatles                                   Beatles For Sale  1964.0   
1  Imagine Dragons                            Infinity Blade II [OST]  2011.0   
2      The Beatles                                          Let It Be  1970.0   
3      The Beatles                         Paperback Writer [Single]   1966.0   
4  Imagine Dragons  Divergente 2 - L'insurrection (The Divergent S...  2015.0   

                 title  number  \
0  Rock and Roll Music     4.0   
1              Monster     3.0   
2            I Me Mine     4.0   
3                 Rain     2.0   
4             Warriors     7.0   

                                    original_version  \
0  chorus\nJust let me hear some of that rock and...   
1  Ever since I could remember\nEverything inside...   
2  All through the day I me mine, I me mine, I me...   
3  If the rain comes they run and hide their head...   
4  Warr

In [8]:
# Créer un tokenizer
tokenizer = Tokenizer()

# Apprentissage du vocabulaire
tokenizer.fit_on_texts(X_train + y_train)

# Convertir les textes en séquences d'indices
X_seq = tokenizer.texts_to_sequences(X_train)
y_seq = tokenizer.texts_to_sequences(y_train)

# Définir la longueur maximale (limiter pour la mémoire)
max_length = 300  # Ajustez cette valeur si nécessaire

# Padding pour uniformiser les séquences
X_padded = pad_sequences(X_seq, maxlen=max_length, padding="post")
y_padded = pad_sequences(y_seq, maxlen=max_length, padding="post")

# Vérification des formes
print("Forme de X_padded :", X_padded.shape)
print("Forme de y_padded :", y_padded.shape)


Forme de X_padded : (99289, 300)
Forme de y_padded : (99289, 300)


In [9]:
# Définir le modèle GRU
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(GRU(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(len(tokenizer.word_index) + 1, activation="softmax"))

# Compilation du modèle
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Afficher la structure du modèle
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)